In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

from lib.data.datareader_total import DataReaderTotal
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.utils.args import get_opts_args
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.data.dataset_motion_3d import MotionDataset3DTotal
from lib.model.evaluation import *

### Load data_type

In [7]:
dataset_name = 'h36m'
overwrite_list = []
source_list                    = load_data(dataset_name=dataset_name, data_type='source_list', overwrite_list=overwrite_list)
cam_params                     = load_data(dataset_name=dataset_name, data_type='cam_param', overwrite_list=overwrite_list)
world_3ds                      = load_data(dataset_name=dataset_name, data_type='world_3d', overwrite_list=overwrite_list)
cam_3ds                        = load_data(dataset_name=dataset_name, data_type='cam_3d', overwrite_list=overwrite_list)
img_2ds                        = load_data(dataset_name=dataset_name, data_type='img_2d', overwrite_list=overwrite_list)
img_3ds                        = load_data(dataset_name=dataset_name, data_type='img_3d', overwrite_list=overwrite_list)
scale_factors                  = load_data(dataset_name=dataset_name, data_type='scale_factor', overwrite_list=overwrite_list)
img_25ds                       = load_data(dataset_name=dataset_name, data_type='img_25d', overwrite_list=overwrite_list)
cam_3d_canonicals_same_z       = load_data(dataset_name=dataset_name, data_type='cam_3d_canonical', canonical_type='same_z', overwrite_list=overwrite_list)
cam_3d_canonicals_same_dist    = load_data(dataset_name=dataset_name, data_type='cam_3d_canonical', canonical_type='same_dist', overwrite_list=overwrite_list)
cam_3d_canonicals_fixed_dist_5 = load_data(dataset_name=dataset_name, data_type='cam_3d_canonical', canonical_type='fixed_dist_5', overwrite_list=overwrite_list)
img_2d_canonicals_same_z       = load_data(dataset_name=dataset_name, data_type='img_2d_canonical', canonical_type='same_z', overwrite_list=overwrite_list)
img_2d_canonicals_same_dist    = load_data(dataset_name=dataset_name, data_type='img_2d_canonical', canonical_type='same_dist', overwrite_list=overwrite_list)
img_2d_canonicals_fixed_dist_5 = load_data(dataset_name=dataset_name, data_type='img_2d_canonical', canonical_type='fixed_dist_5', overwrite_list=overwrite_list)

==> Loading H36M source_list... overwrite: False
==> Loading H36M cam_param... overwrite: False
==> Loading H36M world_3d... overwrite: False
==> Loading H36M cam_3d... overwrite: False
==> Loading H36M img_2d... overwrite: False
==> Loading H36M img_3d... overwrite: False
==> Loading H36M scale_factor... overwrite: False
==> Loading H36M img_25d... overwrite: False
==> Loading H36M cam_3d_canonical_same_z... overwrite: False
==> Loading H36M cam_3d_canonical_same_dist... overwrite: False
==> Loading H36M cam_3d_canonical_fixed_dist_5... overwrite: False
==> Loading H36M img_2d_canonical_same_z... overwrite: False
==> Loading H36M img_2d_canonical_same_dist... overwrite: False
==> Loading H36M img_2d_canonical_fixed_dist_5... overwrite: False


In [3]:
# check constraints
for source in source_list:
    subject, cam_id, action = split_source_name(source, dataset_name)
    world_3d = world_3ds[subject][action]
    cam_3d = cam_3ds[subject][action][cam_id]
    img_2d = img_2ds[subject][action][cam_id]
    img_3d = img_3ds[subject][action][cam_id]
    cam_3d_canonical_same_z = cam_3d_canonicals_same_z[subject][action][cam_id]
    cam_3d_canonical_same_dist = cam_3d_canonicals_same_dist[subject][action][cam_id]
    cam_3d_canonical_fixed_dist_5 = cam_3d_canonicals_fixed_dist_5[subject][action][cam_id]
    assert world_3d.shape[1:] == (17, 3), f'{source} {world_3ds[subject][action].shape}'
    assert world_3d.mean() < 5, f'{source} {world_3d.mean()}'
    assert cam_3d.shape[1:] == (17, 3), f'{source} {cam_3d.shape}'
    assert cam_3d.mean() < 5, f'{source} {cam_3d.mean()}'
    assert img_2d.mean() > 1, f'{source} {img_2d.mean()}'
    assert np.allclose(img_2d, img_3d[...,:2]), f'image 2d and 3d are not equal'
    assert np.allclose(cam_3d[...,2], cam_3d_canonical_same_z[...,2]), 'Z values should be same'
    assert np.allclose(np.linalg.norm(cam_3d[:, 0], axis=1), cam_3d_canonical_same_dist[:, 0, 2]), 'Distances should be same'
    assert np.allclose(np.array([5.0]*len(cam_3d)), cam_3d_canonical_fixed_dist_5[:, 0, 2]), 'Distances should be same'